In [1]:
print("Kernel is up")

Kernel is up


In [5]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
tf.enable_eager_execution()

import librosa
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import librosa
#import simpleaudio as sa
import math
from IPython import display
import sys
import preprocessing.specgrams_helper as preprocessing

print("Finished imports")

Finished imports


In [16]:
# Converting into spectrograms
# Spectrograms are stored into spectrograms.tfrecord

data_per_file = 1920
total_files = 27
nsynth = iter(tf.data.TFRecordDataset("gs://jz-datasets/nsynth/train.tfrecord"))
total_passed = 0
def acousticOnly():
    global total_passed
    i = 0
    just_skipped = False
    while(True):
        try: 
            data = next(nsynth)
        except:
            break
        data = tf.parse_single_example(serialized=data, features=read_features)
        family = data['instrument_family'].numpy();
        if(data['instrument_source'].numpy()==0 and (family in [1, 2, 6, 7, 8])):
            if(just_skipped):
                print("Skipped {}".format(i))
            just_skipped = False
            total_passed += 1
            yield data
        else:
            if(not(just_skipped)):
                print("                                       ", end='\r')
            i += 1
            if(i % 500 == 0):
                print("Skipped {}".format(i), end="\r")
            just_skipped = True

record = acousticOnly()

read_features = {
    'note': tf.FixedLenFeature([], dtype=tf.int64),
    'note_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_str': tf.FixedLenFeature([], dtype=tf.string),
    'pitch': tf.FixedLenFeature([], dtype=tf.int64),
    'velocity': tf.FixedLenFeature([], dtype=tf.int64),
    'sample_rate': tf.FixedLenFeature([], dtype=tf.int64),
    'audio': tf.VarLenFeature(dtype=float),
    'instrument_family': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_family_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument_source': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_source_str': tf.FixedLenFeature([], dtype=tf.string)
}

start_on_file = 0
for i in range(data_per_file*start_on_file):
    if(i % 200 == 0):
        print("Skipped {} out of {}".format(i, data_per_file*start_on_file), end='\r')
    next(record)
if(start_on_file != 0):
    print("Skipped {} out of {}".format(data_per_file*start_on_file, data_per_file*start_on_file))

spec_dim = (128, 1024)
data_helper = preprocessing.SpecgramsHelper(spec_dim[0:2])
    
for i in range(total_files):
    if(i < start_on_file):
        continue
    print("Starting file {}".format(i))
    with tf.python_io.TFRecordWriter('gs://jz-datasets/spec-pruned-files/spectrograms{}.tfrecord'.format(i)) as writer:
        total = 0
        for _ in range(data_per_file):
            total += 1
            data = next(record)
            audio = data['audio'].values
            spec = data_helper.waves_to_melspecgrams(tf.expand_dims(tf.expand_dims(audio, 1), 0))[0].numpy()
            spec = spec.flatten().tolist()
            """.numpy()*15000
            audio = np.array(audio)
            spec = librosa.stft(audio, 254, 127)
            magnitudes = np.log(np.abs(spec) + 1.2664166e-14).reshape(1, 128, 504)
            phases = np.angle(spec).reshape(1, 128, 504)
            magnitudes = 2*(magnitudes+32)/(16+32)-1
            spec = np.concatenate([magnitudes, tf.cos(phases), tf.sin(phases)], 0).flatten().tolist()"""
            example = tf.train.Example(features=tf.train.Features(feature={
                'note': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['note'].numpy()])),
                'note_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['note_str'].numpy()])),
                'instrument': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument'].numpy()])),
                'instrument_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_str'].numpy()])),
                'pitch': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['pitch'].numpy()])),
                'velocity': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['velocity'].numpy()])),
                'sample_rate': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['sample_rate'].numpy()])),
                'spectrogram': tf.train.Feature(float_list=tf.train.FloatList(value=spec)),
                'instrument_family': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_family'].numpy()])),
                'instrument_family_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_family_str'].numpy()])),
                'instrument_source': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_source'].numpy()])),
                'instrument_source_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_source_str'].numpy()]))
            }))

            writer.write(example.SerializeToString())
            if(total % 20 == 0):
                print("Finished {} out of {}".format(total, data_per_file), end='\r')
        print("Finished {} out of {}".format(data_per_file, data_per_file))
        print()
            
print("\nDone")
print("Total: {}".format(total_skipped))

Starting file 0
Skipped 57517                          
Finished 1920 out of 1920

Starting file 1
Finished 1920 out of 1920

Starting file 2
Finished 1920 out of 1920

Starting file 3
Skipped 116399                         
Skipped 160291                         
Finished 1920 out of 1920

Starting file 4
Finished 1920 out of 1920

Starting file 5
Finished 1920 out of 1920

Starting file 6
Finished 1920 out of 1920

Starting file 7
Skipped 166539                         
Finished 1920 out of 1920

Starting file 8
Finished 1920 out of 1920

Starting file 9
Finished 1920 out of 1920

Starting file 10
Finished 1920 out of 1920

Starting file 11
Finished 1920 out of 1920

Starting file 12
Finished 1920 out of 1920

Starting file 13
Finished 1920 out of 1920

Starting file 14
Finished 1920 out of 1920

Starting file 15
Skipped 177400                         
Finished 1920 out of 1920

Starting file 16
Finished 1920 out of 1920

Starting file 17
Finished 1920 out of 1920

Starting file 18
S

NameError: name 'total_skipped' is not defined

In [16]:
!rm .lock

rm: cannot remove '.lock': No such file or directory


In [4]:
print(total_passed)

51840
